In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [4]:
import json
with open("service1906_1506.json") as of:
    data = json.load(of)

In [5]:
computes = [c for c in data.keys() if c!="timespan"]
variables = [v for v in data[computes[0]] if v!='index' and v!='arrJob_scheduling']

In [6]:
#Check empty array
def getEmptyArr(data, c):
    cObj = data[c]
    cDf = pd.DataFrame()
    cDf['compute'] = [c for _ in data['timespan']]
    cDf['timespan'] = data['timespan']
    for v in variables:
        vArr = np.array(cObj[v])
        if len(vArr)==0:
            print('c=', c)
            print('v=', v)
for c in computes:
    getEmptyArr(data, c)

c= compute-1-26
v= arrTemperature


In [7]:
def addTarget(cDf, predictedVar, predictedStep, target):
    cDf[target] = cDf[predictedVar].shift(-predictedStep)
    cDf.dropna(inplace=True)
    return cDf

In [8]:
def getComputeDf(data, c, predictedVar, predictedStep, target):
    cObj = data[c]
    cDf = pd.DataFrame()
    cDf['compute'] = [c for _ in data['timespan']]
    cDf['timespan'] = data['timespan']
    for v in variables:
        vArr = np.array(cObj[v])
        if len(vArr)==0:
            return None
        else:
            for i in range(len(vArr[0])):
                cDf[v+str(i)] = vArr[:, i]
    cDf['timespan'] = pd.to_datetime(cDf['timespan'])
    return addTarget(cDf, predictedVar, predictedStep, target)

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from IPython.display import FileLink
import codecs, json

In [10]:
def exportNPArrayToJson(a, fileName):
    b = a.tolist()
    json.dump(b, codecs.open(fileName, 'w', encoding='utf-8'))

In [11]:
def splitTrainTest(predictedVar):
    target = predictedVar + "_target"
    predictedSteps = 4
    df = pd.concat([x for x in [getComputeDf(data, c, predictedVar, predictedSteps, target) for c in computes] if type(x)!="NoneType"])
    
    df = df.reset_index().drop('index', axis=1)
    features = [x for x in df.columns if x not in ['compute', 'timespan', target]]
        
    X_dfs = []
    y = []
    numberOfSequences = 1
    # generate training data.
    for compute in computes:
        cDf = df[df['compute']==compute]
        if(len(cDf) > sequenceSteps):
            randSteps = np.random.randint(0, len(cDf)-sequenceSteps, numberOfSequences)
            for randStep in randSteps:
                X_dfs.append(cDf.iloc[randStep:randStep+sequenceSteps])
                y.append(X_dfs[-1][target].values[-1])

    X_train_dfs, X_test_dfs, y_train, y_test = train_test_split(X_dfs, y, test_size=0.33)
            
    #Scale data

    # combine the training data to create a scaler
    train_dfs = pd.concat(X_train_dfs)

    scaler = StandardScaler().fit(train_dfs[features].values)
    
    X_train = np.array([scaler.transform(item[features].values) for item in X_train_dfs])
    X_test = np.array([scaler.transform(item[features].values) for item in X_test_dfs])
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    
    exportNPArrayToJson(X_train, 'newData/'+target+'_X_train_HPCC_'+str(sequenceSteps)+'.json')
    exportNPArrayToJson(X_test, 'newData/'+target+'_X_test_HPCC_'+str(sequenceSteps)+'.json')
    exportNPArrayToJson(y_train, 'newData/'+target+'_y_train_HPCC_'+str(sequenceSteps)+'.json')
    exportNPArrayToJson(y_test, 'newData/'+target+'_y_test_HPCC_'+str(sequenceSteps)+'.json')
    
    FileLink('newData/'+target+'_X_train_HPCC_'+str(sequenceSteps)+'.json')
    FileLink('newData/'+target+'_X_test_HPCC_'+str(sequenceSteps)+'.json')
    FileLink('newData/'+target+'_y_train_HPCC_'+str(sequenceSteps)+'.json')
    FileLink('newData/'+target+'_y_test_HPCC_'+str(sequenceSteps)+'.json')
    
    return (X_train, X_test, y_train, y_test)

In [12]:
from keras import regularizers
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import Flatten

# from keras import backend as K
# K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=36, inter_op_parallelism_threads=36)))


def createModel(l1Nodes, l2Nodes, d1Nodes, d2Nodes, inputShape):
    # input layer
    lstm1 = LSTM(l1Nodes, input_shape=inputShape, return_sequences=True, kernel_regularizer=regularizers.l2(0.1))
    do1 = Dropout(0.2)
    
    lstm2 = LSTM(l2Nodes, return_sequences=True, kernel_regularizer=regularizers.l2(0.1))
    do2 = Dropout(0.2)
    
    flatten = Flatten()
    
    dense1 = Dense(d1Nodes, activation='relu')
    do3 = Dropout(0.2)
    
    dense2 = Dense(d2Nodes, activation='relu')
    do4 = Dropout(0.2)
    
    # output layer
    outL = Dense(1, activation='relu')
    # combine the layers
#     layers = [lstm1, do1, lstm2, do2, dense1, do3, dense2, do4, outL]
    layers = [lstm1, lstm2, flatten,  dense1, dense2, outL]
#     layers = [lstm1, flatten,  dense1, outL]

    # create the model
    model = Sequential(layers)
    model.compile(optimizer='adam', loss='mse')
    return model

Using TensorFlow backend.


In [13]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.models import load_model
# ten fold
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, shuffle=True)
from datetime import datetime
# import tensorflowjs as tfjs

sequenceSteps = 100

msescores = []
model_config = "config_8_8_8_4_ts"+str(sequenceSteps)

tempAllVariables = ['arrTemperature0', 'arrTemperature1', 'arrTemperature2', 'arrCPU_load0', 'arrMemory_usage0', 'arrFans_health0', 'arrFans_health1', 'arrFans_health2', 'arrFans_health3', 'arrPower_usage0']
allVariables = ['arrTemperature0']
timeList = []



for feature in allVariables:
    print(feature)
    X_train, X_test, y_train, Y_test = splitTrainTest(feature)

    for training_time in range(0,1):
        start_time = datetime.now()
        print(start_time)

        counter= 0
        for trainIdx, testIdx in kfold.split(X_train, y_train):
            # create callbacks
            model_path = feature+'_training_time_'+str(training_time)+'_best_model_fold_'+str(counter)+"_"+model_config+'.h5'
            mc = ModelCheckpoint('newModel/'+model_path, monitor='val_loss', mode='min', verbose=1, save_best_only=True)
            es = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=1)
            # create model
            model = createModel(l1Nodes = 8, l2Nodes = 8, d1Nodes = 8, d2Nodes = 4, inputShape= (X_train.shape[1], X_train.shape[2]))
            model.fit(X_train[trainIdx], y_train[trainIdx], validation_data=(X_train[testIdx], y_train[testIdx]), batch_size=32, epochs=40, callbacks=[mc, es])
            # Done load the best model of this fold
            saved_model = load_model('newModel/'+model_path)
            msescores.append({'path': 'newModel/'+model_path, 'mse': saved_model.evaluate(X_train[testIdx], y_train[testIdx])})
            counter = counter + 1
        
#             tfjs.converters.save_keras_model(saved_model, 'newModel/'+model_path)

        end_time = datetime.now()
    
        timeList.append({'start': start_time, 'end': end_time, 'time': datetime.timestamp(end_time)-datetime.timestamp(start_time)})

arrTemperature0
2019-11-04 19:17:34.847275
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 249 samples, validate on 63 samples
Epoch 1/40
249/249 [==============================] - 1s 5ms/step - loss: 4019.5911 - val_loss: 3750.1671

Epoch 00001: val_loss improved from inf to 3750.16706, saving model to newModel/arrTemperature0_training_time_0_best_model_fold_0_config_8_8_8_4_ts100.h5
Epoch 2/40
249/249 [==============================] - 0s 2ms/step - loss: 3964.1338 - val_loss: 3672.7190

Epoch 00002: val_loss improved from 3750.16706 to 3672.71896, saving model to newModel/arrTemperature0_training_time_0_best_model_fold_0_config_8_8_8_4_ts100.h5
Epoch 3/40
249/249 [==============================] - 0s 2ms/step - loss: 3864.9169 - val_loss: 3541.9813

Epoch 00003: val_loss improved from 3672.71896 to 3541.98133, saving model to newModel/arrTemperature0_training_time_0_best_model_fold_0_config_8_8_8_4_ts100.h5
Epoch 4/40
249/249 [=============

249/249 [==============================] - 0s 2ms/step - loss: 57.9658 - val_loss: 75.6791

Epoch 00029: val_loss improved from 77.26979 to 75.67907, saving model to newModel/arrTemperature0_training_time_0_best_model_fold_0_config_8_8_8_4_ts100.h5
Epoch 30/40
249/249 [==============================] - 0s 2ms/step - loss: 56.3860 - val_loss: 74.2677

Epoch 00030: val_loss improved from 75.67907 to 74.26768, saving model to newModel/arrTemperature0_training_time_0_best_model_fold_0_config_8_8_8_4_ts100.h5
Epoch 31/40
249/249 [==============================] - 0s 2ms/step - loss: 54.8116 - val_loss: 72.7398

Epoch 00031: val_loss improved from 74.26768 to 72.73983, saving model to newModel/arrTemperature0_training_time_0_best_model_fold_0_config_8_8_8_4_ts100.h5
Epoch 32/40
249/249 [==============================] - 0s 2ms/step - loss: 53.4109 - val_loss: 71.4782

Epoch 00032: val_loss improved from 72.73983 to 71.47825, saving model to newModel/arrTemperature0_training_time_0_best_model

249/249 [==============================] - 0s 2ms/step - loss: 226.4947 - val_loss: 192.5920

Epoch 00020: val_loss improved from 224.23573 to 192.59196, saving model to newModel/arrTemperature0_training_time_0_best_model_fold_1_config_8_8_8_4_ts100.h5
Epoch 21/40
249/249 [==============================] - 0s 2ms/step - loss: 187.1277 - val_loss: 167.6110

Epoch 00021: val_loss improved from 192.59196 to 167.61097, saving model to newModel/arrTemperature0_training_time_0_best_model_fold_1_config_8_8_8_4_ts100.h5
Epoch 22/40
249/249 [==============================] - 0s 2ms/step - loss: 161.0396 - val_loss: 156.1738

Epoch 00022: val_loss improved from 167.61097 to 156.17377, saving model to newModel/arrTemperature0_training_time_0_best_model_fold_1_config_8_8_8_4_ts100.h5
Epoch 23/40
249/249 [==============================] - 0s 2ms/step - loss: 149.8530 - val_loss: 147.5351

Epoch 00023: val_loss improved from 156.17377 to 147.53512, saving model to newModel/arrTemperature0_training_t

250/250 [==============================] - 0s 2ms/step - loss: 566.0969 - val_loss: 626.5826

Epoch 00011: val_loss improved from 707.82517 to 626.58262, saving model to newModel/arrTemperature0_training_time_0_best_model_fold_2_config_8_8_8_4_ts100.h5
Epoch 12/40
250/250 [==============================] - 0s 2ms/step - loss: 510.9726 - val_loss: 566.1634

Epoch 00012: val_loss improved from 626.58262 to 566.16341, saving model to newModel/arrTemperature0_training_time_0_best_model_fold_2_config_8_8_8_4_ts100.h5
Epoch 13/40
250/250 [==============================] - 0s 2ms/step - loss: 441.4303 - val_loss: 487.7021

Epoch 00013: val_loss improved from 566.16341 to 487.70213, saving model to newModel/arrTemperature0_training_time_0_best_model_fold_2_config_8_8_8_4_ts100.h5
Epoch 14/40
250/250 [==============================] - 0s 2ms/step - loss: 372.5927 - val_loss: 413.0562

Epoch 00014: val_loss improved from 487.70213 to 413.05621, saving model to newModel/arrTemperature0_training_t

Epoch 2/40
250/250 [==============================] - 0s 2ms/step - loss: 3782.6942 - val_loss: 3765.1181

Epoch 00002: val_loss improved from 3946.29719 to 3765.11807, saving model to newModel/arrTemperature0_training_time_0_best_model_fold_3_config_8_8_8_4_ts100.h5
Epoch 3/40
250/250 [==============================] - 0s 2ms/step - loss: 3580.9605 - val_loss: 3444.6351

Epoch 00003: val_loss improved from 3765.11807 to 3444.63509, saving model to newModel/arrTemperature0_training_time_0_best_model_fold_3_config_8_8_8_4_ts100.h5
Epoch 4/40
250/250 [==============================] - 0s 2ms/step - loss: 3265.6661 - val_loss: 2955.0411

Epoch 00004: val_loss improved from 3444.63509 to 2955.04109, saving model to newModel/arrTemperature0_training_time_0_best_model_fold_3_config_8_8_8_4_ts100.h5
Epoch 5/40
250/250 [==============================] - 0s 2ms/step - loss: 2782.0679 - val_loss: 2338.6641

Epoch 00005: val_loss improved from 2955.04109 to 2338.66409, saving model to newModel/ar

250/250 [==============================] - 0s 2ms/step - loss: 110.5313 - val_loss: 192.4279

Epoch 00034: val_loss improved from 197.37413 to 192.42793, saving model to newModel/arrTemperature0_training_time_0_best_model_fold_3_config_8_8_8_4_ts100.h5
Epoch 35/40
250/250 [==============================] - 0s 2ms/step - loss: 105.0751 - val_loss: 180.7282

Epoch 00035: val_loss improved from 192.42793 to 180.72823, saving model to newModel/arrTemperature0_training_time_0_best_model_fold_3_config_8_8_8_4_ts100.h5
Epoch 36/40
250/250 [==============================] - 0s 2ms/step - loss: 99.3677 - val_loss: 169.4701

Epoch 00036: val_loss improved from 180.72823 to 169.47013, saving model to newModel/arrTemperature0_training_time_0_best_model_fold_3_config_8_8_8_4_ts100.h5
Epoch 37/40
250/250 [==============================] - 0s 2ms/step - loss: 94.1123 - val_loss: 161.9383

Epoch 00037: val_loss improved from 169.47013 to 161.93827, saving model to newModel/arrTemperature0_training_tim

250/250 [==============================] - 0s 2ms/step - loss: 59.6465 - val_loss: 93.5229

Epoch 00025: val_loss improved from 94.36933 to 93.52286, saving model to newModel/arrTemperature0_training_time_0_best_model_fold_4_config_8_8_8_4_ts100.h5
Epoch 26/40
250/250 [==============================] - 0s 2ms/step - loss: 58.6846 - val_loss: 94.4910

Epoch 00026: val_loss did not improve from 93.52286
Epoch 27/40
250/250 [==============================] - 0s 2ms/step - loss: 57.8573 - val_loss: 92.9746

Epoch 00027: val_loss improved from 93.52286 to 92.97459, saving model to newModel/arrTemperature0_training_time_0_best_model_fold_4_config_8_8_8_4_ts100.h5
Epoch 28/40
250/250 [==============================] - 0s 2ms/step - loss: 57.0162 - val_loss: 90.5180

Epoch 00028: val_loss improved from 92.97459 to 90.51796, saving model to newModel/arrTemperature0_training_time_0_best_model_fold_4_config_8_8_8_4_ts100.h5
Epoch 29/40
250/250 [==============================] - 0s 2ms/step - loss

In [14]:
print ('start time: ',  datetime.timestamp(start_time))
print ('end time:', datetime.timestamp(end_time))
print ('training time: ', datetime.timestamp(end_time)-datetime.timestamp(start_time))

start time:  1572916654.847275
end time: 1572916789.971276
training time:  135.12400102615356


In [15]:
msescores

[{'path': 'newModel/arrTemperature0_training_time_0_best_model_fold_0_config_8_8_8_4_ts100.h5',
  'mse': 62.702007354251926},
 {'path': 'newModel/arrTemperature0_training_time_0_best_model_fold_1_config_8_8_8_4_ts100.h5',
  'mse': 95.65364886087085},
 {'path': 'newModel/arrTemperature0_training_time_0_best_model_fold_2_config_8_8_8_4_ts100.h5',
  'mse': 99.30110537621283},
 {'path': 'newModel/arrTemperature0_training_time_0_best_model_fold_3_config_8_8_8_4_ts100.h5',
  'mse': 143.98928316177862},
 {'path': 'newModel/arrTemperature0_training_time_0_best_model_fold_4_config_8_8_8_4_ts100.h5',
  'mse': 81.0743162093624}]

In [16]:
# timeList
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 100, 8)            608       
_________________________________________________________________
lstm_10 (LSTM)               (None, 100, 8)            544       
_________________________________________________________________
flatten_5 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 8)                 6408      
_________________________________________________________________
dense_14 (Dense)             (None, 4)                 36        
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 5         
Total params: 7,601
Trainable params: 7,601
Non-trainable params: 0
____________________________________________________

In [ ]:
# Plot and see data distribution

from matplotlib import pyplot as plt
import seaborn as sns

def plotAttrDataOfId(data, compute, features):
    plt.figure(figsize=(30, 20))
    for i, v in enumerate(features):
        plt.subplot(10, 3, i+1)
        cDf = df[df['compute']==compute]
        plt.plot(cDf['timespan'], cDf[v])
        plt.title(v)
        plt.tight_layout()

In [ ]:
# for x in np.random.randint(0, len(computes), 3):
#     plotAttrDataOfId(df, computes[x], features)

In [ ]:
def plotDataDistribution(data, features):
    plt.figure(figsize=(30, 10))
    for i, v in enumerate(features):
        plt.subplot(3, 10, i+1)
        sns.distplot(list(data[v].values))
        plt.title(v)
    plt.tight_layout()

In [ ]:
# plotDataDistribution(df, features)

In [ ]:
from datetime import datetime
print(datetime.now().strftime("%H_%M_%S"))

In [ ]:
[i for i in range(1,11
                 )]

In [ ]:
sns.distplot(y_train)

In [ ]:
sns.distplot(y_test)

In [ ]:
print (type(X_dfs[0]))